In [1]:
%load_ext autoreload
%autoreload 2

from gpu import set_gpu
import numpy as np
import os
import adapt.utils.data_utils as prd
import adapt.loop as lp
import adapt.ml.lda as dlda
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import adapt.ml.dl_subclass as dl
import copy as cp
import tensorflow as tf
import pickle
from datetime import datetime
import time
from IPython import display

set_gpu()
import scipy as scipy

Num GPUs Available:  1


In [2]:
struct = scipy.io.loadmat('C:/Users/yteh/Documents/work/git/projects/adaptive/matlab/ARtest2.mat')
daq = struct['data']
feat = struct['feat']

In [13]:
order = 6
data = daq[:1,0,...]
samp = data.shape[0]
data = data.astype('float32')*10/(2**16-1)-5
AR = np.zeros((order+1,samp))
K = np.zeros((order+1,samp))
AR[0,:] = 1
R0 = np.sum(np.multiply(data,data),axis=1)
R = np.zeros((samp,order))
for i in range(order):
    R[:,i] = np.sum(np.multiply(data[:,:-1*(i+1)],data[:,i+1:]),axis=1)
E = cp.deepcopy(R0)
AR[1,:] = -R[:,0]/R0
K[0,:] = AR[1,:]
q = R[:,0]
tmp = np.zeros((samp,order))
i = 0
for i in range(1):
    E += np.multiply(q,K[i,:].T)
    q = R[:,i+1]
    S = np.zeros((samp,))
    for k in range(i+1):
        S[:] += np.multiply(R[:,k],AR[i+1-k,:].T)
    q += S
    K[i+1,:] = -q/E
    for k in range(i+1):
        tmp[:,k] = np.multiply(K[i+1,:],AR[i+1-k,:])

    for k in range(1,i+2):
        AR[k,:] = AR[k,:] + tmp[:,k-1]

    AR[i+2,:] = K[i+1,:]

In [22]:
order = 6
data = daq[:1,0,...]
samp = data.shape[0]
data = data.astype('float32')*10/(2**16-1)-5
AR = np.zeros((order+1,samp))
K = np.zeros((order+1,samp))
AR[0,:] = 1
R0 = np.sum(np.multiply(data,data),axis=1)
R = np.zeros((samp,order))
for i in range(order):
    R[:,i] = np.sum(np.multiply(data[:,:-1*(i+1)],data[:,i+1:]),axis=1)
E = cp.deepcopy(R0)
AR[1,:] = -R[:,0]/R0
K[0,:] = AR[1,:]
q = cp.deepcopy(R[:,0])
tmp = np.zeros((samp,order))

for i in range(1):
    E += np.multiply(q,K[i,:].T)
    q = cp.deepcopy(R[:,i+1])
    S = np.zeros((samp,))
    for k in range(i+1):
        print(AR[i+1-k,:])
        print(R[:,k])
        S[:] += np.multiply(R[:,k],AR[i+1-k,:].T)

    q += S
    K[i+1,:] = -q/E
    for k in range(i+1):
        tmp[:,k] = np.multiply(K[i+1,:],AR[i+1-k,:])

    for k in range(1,i+2):
        AR[k,:] = AR[k,:] + tmp[:,k-1]

    AR[i+2,:] = K[i+1,:]

In [ ]:
E += q*K[i,0]
    q = R[0,i+1]
    S = 0
    for k in range(i+1):
        S += R[0,k] * AR[i+1-k,0]
    q += S
    K[i+1,0] = -q/E
    for k in range(i+1):
        tmp[0,k] = K[i+1,0] * AR[i+1-k,0]

    for k in range(1,i+2):
        AR[k,0] = AR[k,0] + tmp[0,k-1]

    AR[i+2,0] = K[i+1,0]